# Prerequisite

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pickle
# Load 10,000 unmon pickle file
print("Loading unmon_standard...")
with open('/content/drive/MyDrive/GitHub/global/dataset/unmon_standard10.pkl', 'rb') as f:  # Path to unmon_standard10.pkl in Colab
    unmon_data = pickle.load(f)

Loading unmon_standard...


In [ ]:
print(type(unmon_data))
print(len(unmon_data)) #10000 observation
print(len(unmon_data[1000])) # timestamps with direction, not equals between each observations



---


# Dataframe

- sequence of packet sizes
- sequence of packet time stamps
- packet size, direction
- sequence of cumluative packet sizes
- sequence of bursts

- number of packet, total incoming, total outgoing

incoming : server to client
outcoming : client to server

## unmonitored

In [3]:
import pandas as pd
TOTAL_URLS = 10000  # total number in the dataset

unmon_time = [] # Array to store instances (timestamps) - 10,000 instances, e.g., [[0.0, 0.5, 3.4, ...], [0.0, 4.5, ...], [0.0, 1.5, ...], ... [... ,45.8]]
unmon_size = [] # Array to store instances (direction*size) - size information
unmon_cumul = []
unmon_burst = []

for i in range(TOTAL_URLS):
    size_seq = []
    time_seq = []
    cumul_seq = []
    burst_seq = []
    sum = 0
    for c in unmon_data[i]:
        dr = 1 if c > 0 else -1
        time_seq.append(abs(c))
        size_seq.append(dr * 512) # In the pickle file, there is no size information, so the conversion code is set to multiply by 512 uniformly.
        sum += dr * 512
        cumul_seq.append(sum)

    unmon_time.append(time_seq)
    unmon_size.append(size_seq)
    unmon_cumul.append(cumul_seq)

    burst = 1
    for j in range(len(size_seq)-1):
        if size_seq[j] == size_seq[j+1]:
          burst += 1
        elif j == len(size_seq)-1:
          burst_seq.append(size_seq[j] * burst)
          burst = 1
        else:
          burst_seq.append(size_seq[j] * burst)
          burst = 1

    unmon_burst.append(burst_seq)

zipped = pd.DataFrame(zip(unmon_time,unmon_size,[-1]*len(unmon_time),unmon_cumul,unmon_burst),columns=['time_seq','size_seq','label','cumlative_seq','burst_seq'])

print('finish')

Total samples: 10000
10000


In [12]:
zipped

,time_seq,size_seq,label,cumlative_seq,burst_seq
0,"[0.0, 0.13, 0.13, 0.36, 0.83, 1.1, 1.1, 1.35, ...","[-512, -512, 512, -512, 512, -512, 512, -512, ...",-1,"[-512, -1024, -512, -1024, -512, -1024, -512, ...","[-1024, 512, -512, 512, -512, 512, -1024, 1536..."
1,"[0.0, 0.21, 0.21, 0.39, 0.85, 1.36, 1.36, 1.58...","[-512, -512, 512, -512, 512, -512, 512, -512, ...",-1,"[-512, -1024, -512, -1024, -512, -1024, -512, ...","[-1024, 512, -512, 512, -512, 512, -1024, 1024..."
2,"[0.0, 0.23, 0.24, 0.61, 1.84, 2.28, 2.29, 2.69...","[-512, -512, 512, -512, 512, -512, 512, -512, ...",-1,"[-512, -1024, -512, -1024, -512, -1024, -512, ...","[-1024, 512, -512, 512, -512, 512, -1024, 1536..."
3,"[0.0, 0.14, 0.14, 0.31, 0.85, 1.37, 1.37, 1.73...","[-512, -512, 512, -512, 512, -512, 512, -512, ...",-1,"[-512, -1024, -512, -1024, -512, -1024, -512, ...","[-1024, 512, -512, 512, -512, 512, -1024, 512,..."
4,"[0.0, 0.12, 0.12, 0.26, 0.83, 2.56, 2.56, 3.05...","[-512, -512, 512, -512, 512, -512, 512, -512, ...",-1,"[-512, -1024, -512, -1024, -512, -1024, -512, ...","[-1024, 512, -512, 512, -512, 512, -25600, 512..."
...,...,...,...,...,...
9995,"[0.0, 0.17, 0.17, 0.35, 0.63, 0.97, 0.97, 1.75...","[-512, -512, 512, -512, 512, -512, 512, -512, ...",-1,"[-512, -1024, -512, -1024, -512, -1024, -512, ...","[-1024, 512, -512, 512, -512, 512, -512, 512, ..."
9996,"[0.0, 0.79, 0.8, 1.21, 1.54, 2.65, 2.65, 3.47,...","[-512, -512, 512, -512, 512, -512, 512, -512, ...",-1,"[-512, -1024, -512, -1024, -512, -1024, -512, ...","[-1024, 512, -512, 512, -512, 512, -512, 512, ..."
9997,"[0.0, 0.13, 0.13, 0.29, 0.35, 0.77, 0.77, 1.71...","[-512, -512, 512, -512, 512, -512, 512, -512, ...",-1,"[-512, -1024, -512, -1024, -512, -1024, -512, ...","[-1024, 512, -512, 512, -512, 512, -4608, 4096..."
9998,"[0.0, 0.14, 0.14, 0.28, 0.86, 1.15, 1.15, 1.38...","[-512, -512, 512, -512, 512, -512, 512, -512, ...",-1,"[-512, -1024, -512, -1024, -512, -1024, -512, ...","[-1024, 512, -512, 512, -512, 512, -2560, 512,..."


In [13]:
cd /content/drive/MyDrive/GitHub/global

/content/drive/MyDrive/GitHub/global


In [14]:
picklefile = open('unmon_df.pkl','wb')
pickle.dump(zipped,picklefile)
picklefile.close()